# Импорт библиотек

In [ ]:
!pip install comet-ml

import comet_ml
from comet_ml import Experiment

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install pytorch_lightning==1.9.4

In [ ]:
import pytorch_lightning as pl

logger = pl.loggers.comet.CometLogger(
    #secret logs :)
)

In [ ]:
!pip install nemo_toolkit[all]

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

In [ ]:
BRANCH = 'r1.0.0b3'

!pip install pydub
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install omegaconf
!pip install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import os
import json
from google.colab import drive
from omegaconf import OmegaConf

import wave
import torch
import librosa
import torchaudio
import librosa.display
import soundfile as sf
from scipy import signal
import IPython.display as ipd
from pydub import AudioSegment
import torchaudio.functional as F 
from torchaudio.utils import download_asset

import numpy as np
import pandas as pd

import shutil
from google.colab import files

# Загрузка файлов

In [ ]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
!mkdir '/content/dataset'

!mkdir '/content/dataset/train'
!mkdir '/content/dataset/test'
!mkdir '/content/dataset/dev'

!mkdir '/content/dataset/far'

In [ ]:
!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/train.zip' -d '/content/dataset/'
!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/dev.zip' -d '/content/dataset/dev'
!tar -xvf '/content/gdrive/MyDrive/Colab Notebooks/neymark/v0.02_test.tar' -C '/content/dataset/test'

!unzip '/content/gdrive/MyDrive/Colab Notebooks/neymark/farph.zip' -d '/content/dataset/far/' #это для false alarm rate

# Создание .json файлов для хранения путей к аудиозаписям

In [ ]:
import wave

def extract_paths(file_path, outpath):
    labels = ['marvin', 'unknow']
    out = []

    for root, dirs, files in os.walk(file_path):
        for i in dirs:
            for root2, dirs2, files2 in os.walk(file_path+i):
                for j in files2:
                    path = file_path + i + '/' + j

                    duration = 0.0
                    with wave.open(path) as mywav:
                            duration = mywav.getnframes() / mywav.getframerate()

                    command = labels[0] if i == labels[0] else labels[1]
                    
                    out.append({"audio_filepath": path, "duration": duration, 
                                "command": command})


    with open(outpath, 'w') as f:
        for val in out:
            json.dump(val, f)
            f.write("\n")
            f.flush()

In [ ]:
extract_paths('/content/dataset/dev/', '/content/dev.json')
extract_paths('/content/dataset/test/', '/content/test.json')
extract_paths('/content/dataset/train/', '/content/train.json')

extract_paths('/content/dataset/far/', '/content/far.json')

In [ ]:
!mkdir jsons_files
!mv dev.json jsons_files
!mv test.json jsons_files
!mv train.json jsons_files

In [ ]:
shutil.make_archive("/content/jsons_files", "zip", "/content/jsons_files")
!cp /content/jsons_files.zip "/content/gdrive/MyDrive/Colab Notebooks/neymark"
files.download('/content/jsons_files.zip')

# Скачиваем yaml файл с гитхаба и меняем его под наш датасет

In [ ]:
MODEL_CONFIG = "matchboxnet_3x1x64_v2.yaml"

if not os.path.exists(f"configs/{MODEL_CONFIG}"):
  !wget -P configs/ "https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/{MODEL_CONFIG}"

# Создаем переменные, хранящие пути к файлам

In [ ]:
train_dataset = os.path.join('/content/train.json')
val_dataset = os.path.join('/content/dev.json')
test_dataset = os.path.join('/content/test.json')

In [ ]:
far_dataset = os.path.join('/content/far.json')

In [ ]:
!head -n 5 {train_dataset}

{"audio_filepath": "/content/dataset/train/off/72242187_nohash_3.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/82e7204d_nohash_2.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/f7879738_nohash_0.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/de89e2ca_nohash_5.wav", "duration": 1.0, "command": "unknow"}
{"audio_filepath": "/content/dataset/train/off/25132942_nohash_2.wav", "duration": 1.0, "command": "unknow"}


# Конфигурация модели

In [ ]:
config_path = f"{MODEL_CONFIG}"
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

name: MatchboxNet-3x1x64-v2
model:
  sample_rate: 22050
  timesteps: 128
  repeat: 1
  dropout: 0.0
  kernel_size_factor: 1.0
  labels_full:
  - marvin
  - unknow
  labels: ${model.labels_full}
  train_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: true
  validation_ds:
    manifest_filepath: ???
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: false
    val_loss_idx: 0
  test_ds:
    manifest_filepath: null
    sample_rate: ${model.sample_rate}
    labels: ${model.labels}
    batch_size: 128
    shuffle: false
    test_loss_idx: 0
  preprocessor:
    cls: nemo.collections.asr.modules.AudioToMFCCPreprocessor
    params:
      window_size: 0.025
      window_stride: 0.01
      window: hann
      n_mels: 64
      n_mfcc: 64
      n_fft: 512
  spec_augment:
    cls: nemo.collections.asr.modules.SpectrogramAugmentation
    params:
      freq_masks: 2
      ti

In [ ]:
labels = config.model.labels
sample_rate = 22050

In [ ]:
config.model.train_ds.manifest_filepath = train_dataset
config.model.validation_ds.manifest_filepath = val_dataset
config.model.test_ds.manifest_filepath = test_dataset

# Подготовка к обучению

In [ ]:
cuda = 1 if torch.cuda.is_available() else 0

config.trainer.max_epochs = 20
config.trainer.accelerator = 'gpu'

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filename='{epoch}-{val_loss:.2f}-{other_metric:.2f}',
    save_top_k=2,
    verbose=True,
    monitor='val_loss',
    mode='min',
)

Тут я решила составить свой конфиг, так мне показалось быстрее

In [ ]:
conf = {'max_epochs': 20, 'num_nodes': 1, 'accelerator': 'gpu', 
        'accumulate_grad_batches': 1, 'callbacks': checkpoint_callback,
        'log_every_n_steps': 1, 'val_check_interval': 0.25, 'logger': logger}

In [ ]:
trainer = pl.Trainer(**conf)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
asr_model = nemo_asr.models.EncDecClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2023-04-17 07:37:07 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 07:37:07 collections:301] Dataset loaded with 166930 items, total duration of  46.19 hours.
[NeMo I 2023-04-17 07:37:07 collections:303] # 166930 files loaded accounting to # 2 labels
[NeMo I 2023-04-17 07:37:08 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 07:37:08 collections:301] Dataset loaded with 19692 items, total duration of  5.40 hours.
[NeMo I 2023-04-17 07:37:08 collections:303] # 19692 files loaded accounting to # 2 labels
[NeMo I 2023-04-17 07:37:08 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-04-17 07:37:08 collections:301] Dataset loaded with 4890 items, total duration of  1.36 hours.
[NeMo I 2023-04-17 07:37:08 collections:303] # 4890 files loaded accounting to # 2 labels


# Обучение

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

In [ ]:
# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
trainer.fit(asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-04-17 07:37:20 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.95, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )
[NeMo I 2023-04-17 07:37:20 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.PolynomialHoldDecayAnnealing object at 0x7ff6f16c4820>" 
    will be used during training (effective maximum steps = 26100) - 
    Parameters : 
    (power: 2.0
    warmup_ratio: 0.05
    hold_ratio: 0.45
    min_lr: 0.001
    last_epoch: -1
    max_steps: 26100
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                             | Params
-----------------------------------------------------------------------
0 | spec_augmentation | SpectrogramAugmentation          | 0     
1 | crop_or_pad       | CropOrPadSpectrogramAugmentation | 0     
2 | preprocessor      | AudioToMFCCPreprocessor          | 0     
3 | encoder           | ConvASREncoder                   | 73.3 K
4 | decoder           | ConvASRDecoderClassification     | 258   
5 | loss              | CrossEntropyLoss                 | 0     
6 | _accuracy         | TopKClassificationAccuracy       | 0     
-----------------------------------------------------------------------
73.6 K    Trainable params
0         Non-trainable params
73.6 K    Total params
0.294     Total estimated model params size (MB)
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 25447: 'val_loss' was not in top 2


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 25773: 'val_loss' reached 0.01350 (best 0.01350), saving model to 'logs/asr-keyword-spotting/d266e964aba74093b4ca0b4d0a988f2d/checkpoints/epoch=19-val_loss=0.01-other_metric=0.00.ckpt' as top 2


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 26099: 'val_loss' was not in top 2
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/annanet/asr-keyword-spotting/d266e964aba74093b4ca0b4d0a988f2d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     global_step [26100]                   : (0.0, 26099.0)
COMET INFO:     learning_rate [26100]                 : (3.828483750112355e-05, 0.05000000074505806)
COMET INFO:     loss [2610]                           : (0.0018828936154022813, 0.7086796760559082)
COMET 

## Смотрим на тестовую выборку

In [ ]:
trainer.test(asr_model, ckpt_path=None)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/asr-keyword-spotting/d266e964aba74093b4ca0b4d0a988f2d



Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_epoch_top@1      │    0.9983640313148499     │
│         test_loss         │   0.010287849232554436    │
└───────────────────────────┴───────────────────────────┘

COMET WARNING: Couldn't retrieve Google Colab notebook content
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/annanet/asr-keyword-spotting/d266e964aba74093b4ca0b4d0a988f2d
COMET INFO:   Metrics:
COMET INFO:     test_epoch_top@1 : 0.9983640313148499
COMET INFO:     test_loss        : 0.010287849232554436
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:     Name         : pytorch_lightning_4
COMET INFO:   Parameters:
COMET INFO:     cfg/crop_or_pad_augment/_target_     : nemo.collections.asr.modules.CropOrPadSpectrogramAugmentation
COMET INFO:     cfg/crop_or_pad_augment/audio_length : 128
COMET INFO:     cfg/decoder/_target_          

[{'test_loss': 0.010287849232554436, 'test_epoch_top@1': 0.9983640313148499}]